# Batch Optimization Examples

This notebook shows how to run multiple metric/optimizer jobs in a batch using `py_scripts/optimization_workflow.py`.
It also saves audio and plots with a consistent naming scheme matching `save_wav(add_info=True, add_time=True)`.

In [1]:
# Setup and target loading
import numpy as np
import pandas as pd
from IPython.display import display
from pathlib import Path

# Reuse the same configuration as test.ipynb
SR = 44100
DURATION = 2.0
FFT_PAD = 2
FADE_IN_MS = 10.0
FADE_OUT_MS = 10.0

# Load target partials (example: cello)
target_path = 'tsv/cello_single.tsv'
df = pd.read_csv(target_path, sep="\t")
display(df)
target_freqs = df["Frequency (Hz)"].to_numpy()
target_amps = (df["Amplitude"] / df["Amplitude"].max()).to_numpy()
# Extract a short name for naming (e.g., 'cello_single')
target_name = Path(target_path).stem


,Frequency (Hz),Amplitude
0,73.684211,0.005356
1,136.842105,0.004395
2,147.368421,0.102953
3,223.684211,0.011307
4,297.368421,0.035448
5,371.052632,0.005977
6,442.105263,0.002880
7,447.368421,0.006474
8,521.052632,0.011020
9,594.736842,0.002818


## Define batch jobs
- `method`: one of `'de'` (Differential Evolution), `'da'` (Dual Annealing), `'bh'` (Basin Hopping).
- `metric`: any supported metric: `'pearson'`, `'mfcc'`, `'itakura_saito'`, `'spectral_convergence'`, `'cosine'`, `'euclidean'`, `'manhattan'`, `'kl'`.
- `kwargs`: optimizer-specific settings (e.g., `maxiter`, `workers`, `stepsize`).

In [2]:
from py_scripts.optimization_workflow import run_batch_jobs, BatchJob

jobs = [
    BatchJob(method='de', metric='pearson', kwargs={'maxiter': 500, 'workers': -1}),
    BatchJob(method='da', metric='pearson', kwargs={'maxiter': 500}),
    BatchJob(method='bh', metric='pearson', kwargs={'maxiter': 500, 'stepsize': 0.4}),
    # Add more combinations as needed, e.g.:
    # BatchJob(method='de', metric='mfcc', kwargs={'maxiter': 200, 'workers': -1}),
]

rows = run_batch_jobs(
    target_freqs, target_amps, jobs,
    target_name=target_name,
    sr=SR, duration=DURATION, fft_pad=FFT_PAD,
    fade_in_ms=FADE_IN_MS, fade_out_ms=FADE_OUT_MS,
    seed=42,
)

# Summarize batch results (params/history omitted for brevity)
summary = pd.DataFrame([{k: v for k, v in r.items() if k not in ('params','history') } for r in rows])
display(summary.sort_values('best'))


FM 4-osc DE → Pearson:   0%|           0/500 [ETA: ?, Elapsed: 00:00]

[save_wav] wrote: c:\Users\egorp\Nextcloud\code\public_repos\FFTimbre\rendered_audio\optimized_output_fm_cello_single_de_pearson_20250830-125733.wav
TSV file was saved at: c:\Users\egorp\Nextcloud\code\public_repos\FFTimbre\tsv\final_values_fm_cello_single_de_pearson_20250830-125733.tsv


FM 4-osc DA → Pearson:   0%|           0/500 [ETA: ?, Elapsed: 00:00]

[save_wav] wrote: c:\Users\egorp\Nextcloud\code\public_repos\FFTimbre\rendered_audio\optimized_output_fm_cello_single_da_pearson_20250830-125814.wav
TSV file was saved at: c:\Users\egorp\Nextcloud\code\public_repos\FFTimbre\tsv\final_values_fm_cello_single_da_pearson_20250830-125814.tsv


FM 4-osc BH → Pearson:   0%|           0/500 [ETA: ?, Elapsed: 00:00]

[save_wav] wrote: c:\Users\egorp\Nextcloud\code\public_repos\FFTimbre\rendered_audio\optimized_output_fm_cello_single_bh_pearson_20250830-125918.wav
TSV file was saved at: c:\Users\egorp\Nextcloud\code\public_repos\FFTimbre\tsv\final_values_fm_cello_single_bh_pearson_20250830-125918.tsv


,method,metric,best,wav_path,tsv_path,plots
0,de,pearson,0.116531,rendered_audio\optimized_output_fm_cello_singl...,tsv/final_values_fm_cello_single_de_pearson_20...,{'time': 'rendered_plots\optimized_output_fm_c...
1,da,pearson,0.182808,rendered_audio\optimized_output_fm_cello_singl...,tsv/final_values_fm_cello_single_da_pearson_20...,{'time': 'rendered_plots\optimized_output_fm_c...
2,bh,pearson,0.978920,rendered_audio\optimized_output_fm_cello_singl...,tsv/final_values_fm_cello_single_bh_pearson_20...,{'time': 'rendered_plots\optimized_output_fm_c...


## Outputs and naming schema
Each job saves: 
- Audio (`.wav`) to `rendered_audio/optimized_output_fm_<target>_<method>_<metric>_<YYYYMMDD-HHMMSS>.wav`
- Plots (`.png`) to `rendered_plots/optimized_output_fm_<target>_<method>_<metric>_<YYYYMMDD-HHMMSS>_<type>.png`
  - `<type>` ∈ `time`, `spectrum`, `error`
- Final oscillator values to `tsv/final_values_fm_<target>_<method>_<metric>_<YYYYMMDD-HHMMSS>.tsv`

The naming follows the same `_method_metric_timestamp` pattern used by `save_wav(add_info=True, add_time=True)` for consistency across assets.

## Parallel processing note
- `workers` (parallel execution) is supported by Differential Evolution (`method='de'`).
- Dual Annealing (`'da'`) and Basin Hopping (`'bh'`) do not accept `workers`. Any unsupported kwargs are safely ignored by the batch runner.